In [152]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
import numpy as np
from tqdm import tqdm

In [239]:
from __future__ import division

In [454]:
df = pd.read_csv("020328.050012EST.csv")

In [455]:
df["srcAddr1"], df["srcAddr2"], df["srcAddr3"], df["srcAddr4"] = df["Source"].str.split(".", 3).str
df["destAddr1"], df["destAddr2"], df["destAddr3"], df["destAddr4"] = df["Destination"].str.split(".", 3).str
df = df.drop(["Source", "Destination"], axis=1)
# df = df.drop(["firstSize", "secondSize", "threeSize"], axis=1)

In [456]:
df

,srcPort,destPort,firstSize,secondSize,threeSize,flowSize,srcAddr1,srcAddr2,srcAddr3,srcAddr4,destAddr1,destAddr2,destAddr3,destAddr4
0,80,51084,66,64,1438,4767,130,57,89,66,190,84,172,115
1,80,51243,66,64,1518,11266,135,110,241,192,190,84,172,115
2,80,51244,66,1518,1351,2935,135,110,241,192,190,84,172,115
3,80,51248,66,1518,1518,4620,135,110,241,192,190,84,172,115
4,80,51249,66,1518,1167,2751,135,110,241,192,190,84,172,115
5,80,51250,66,1518,1309,2893,135,110,241,192,190,84,172,115
6,80,51254,66,1518,1337,2921,135,110,241,192,190,84,172,115
7,80,51256,66,1518,1122,2706,135,110,241,192,190,84,172,115
8,80,51257,66,1518,1290,2874,135,110,241,192,190,84,172,115
9,80,51258,66,1518,1285,2869,135,110,241,192,190,84,172,115


function to convert data to binary to feed to our neural network

In [441]:
# bits: number of bits, n: number to convert
getBits = lambda bits: lambda n: pd.Series(list(('{0:0%db}'%bits).format(int(n))))

new dataframe for features in binary:

In [442]:
dfb = pd.DataFrame()

In [498]:
SP_cols = ['SP%d'%i for i in range(16)]
dfb[SP_cols] = df['srcPort'].apply(getBits(16))

DP_cols = ['DP%d'%i for i in range(16)]
dfb[DP_cols] = df['destPort'].apply(getBits(16))

for j in range(4):
    SA_cols = ['SA%d'%(i+j*8) for i in range(8)]
    dfb[SA_cols] = df['srcAddr%d'%(j+1)].apply(getBits(8))

for j in range(4):
    DA_cols = ['DA%d'%(i+j*8) for i in range(8)]
    dfb[DA_cols] = df['destAddr%d'%(j+1)].apply(getBits(8))

Pkt1_cols = ['Pkt1-%d'%i for i in range(33)]
dfb[Pkt1_cols] = df['firstSize'].apply(getBits(33))

Pkt2_cols = ['Pkt2-%d'%i for i in range(33)]
dfb[Pkt1_cols] = df['secondSize'].apply(getBits(33))

Pkt3_cols = ['Pkt3-%d'%i for i in range(33)]
dfb[Pkt1_cols] = df['threeSize'].apply(getBits(33))

dfb

,SP0,SP1,SP2,SP3,SP4,SP5,SP6,SP7,SP8,SP9,SP10,SP11,SP12,SP13,SP14,SP15,DP0,DP1,DP2,DP3,DP4,DP5,DP6,DP7,DP8,DP9,DP10,DP11,DP12,DP13,DP14,DP15,SA0,SA1,SA2,SA3,SA4,SA5,SA6,SA7,SA8,SA9,SA10,SA11,SA12,SA13,SA14,SA15,SA16,SA17,SA18,SA19,SA20,SA21,SA22,SA23,SA24,SA25,SA26,SA27,SA28,SA29,SA30,SA31,DA0,DA1,DA2,DA3,DA4,DA5,DA6,DA7,DA8,DA9,DA10,DA11,DA12,DA13,DA14,DA15,DA16,DA17,DA18,DA19,DA20,DA21,DA22,DA23,DA24,DA25,DA26,DA27,DA28,DA29,DA30,DA31,Pkt1-0,Pkt1-1,Pkt1-2,Pkt1-3,Pkt1-4,Pkt1-5,Pkt1-6,Pkt1-7,Pkt1-8,Pkt1-9,Pkt1-10,Pkt1-11,Pkt1-12,Pkt1-13,Pkt1-14,Pkt1-15,Pkt1-16,Pkt1-17,Pkt1-18,Pkt1-19,Pkt1-20,Pkt1-21,Pkt1-22,Pkt1-23,Pkt1-24,Pkt1-25,Pkt1-26,Pkt1-27,Pkt1-28,Pkt1-29,Pkt1-30,Pkt1-31,Pkt1-32
0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,1,1,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,1,1,1,0
2,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,1
3,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,1,1,1,0
4,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,1,1
5,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,1,0,1
6,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,1,1,0,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0,0,1
7,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0
8,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0
9,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,1,1,0,1,0,1,0,0,0,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1


### Training the MLP

convert to matrix

In [531]:
X = dfb.as_matrix()
X[X=='0'] = -1
X[X=='1'] = 1

yr = df["flowSize"]

thres = 1e6
yc = yr.copy(deep=True)
yc[yr > thres] = 1
yc[yr < thres] = -1

In [532]:
sum(yc==-1), sum(yc==1)

(6702, 39)

#### Classifier

In [837]:
X_train, X_test, y_train, y_test = train_test_split(X, yc)

In [838]:
# oversampling minority class
while(sum(y_train==-1)/sum(y_train==1) > 1.2):
    mask = (y_train==1)
    X_train = np.concatenate((X_train, X_train[mask]), axis=0)
    y_train = np.concatenate((y_train, y_train[mask]), axis=0)

In [839]:
sum(y_train==1), sum(y_train==-1), sum(y_test==1), sum(y_test==-1)

(7168, 5027, 11, 1675)

In [840]:
mlp = MLPClassifier(hidden_layer_sizes=(60, 40), activation='tanh', alpha=0.0001)

In [841]:
mlp.fit(X_train,y_train)
predictions = mlp.predict(X_test)

In [842]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[1672    3]
 [   4    7]]
             precision    recall  f1-score   support

         -1       1.00      1.00      1.00      1675
          1       0.70      0.64      0.67        11

avg / total       1.00      1.00      1.00      1686



#### Regression

In [752]:
# X_train, X_test, y_train, y_test = train_test_split(X, yr)